# Saturation Limits for Direct Imaging

For each NIRCam filter, find the saturation limit for a 2-frame (CDS) ramp.

This notebook cycles over all SW and LW filters to find saturation limits for various detector readout modes:
 * Full Frame
 * Window
  * SUB64
  * SUB160
  * SUB320
  * SUB400
  * SUB640
 * Stripe modes
  * STRIPE64
  * STRIPE128
  * STRIPE256
 
Saturation limits for both NIRCam modules (A and B) are calculated.

Results are returned an AstroPy Table of the results, saved in a dictionary `tbls_dict`.

In [1]:
## Standard imports

# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'none'

# seaborn package for making pretty plots, but not necessary
try:
    import seaborn as sns
    params =   {'xtick.direction': 'in', 'ytick.direction': 'in', 'font.family': ['serif'],
                'text.usetex': True}#, 'text.latex.preamble': ['\usepackage{gensymb}']}
    sns.set_style("ticks", params)
except ImportError:
    print('Seaborn module is not installed. Not strictly required, but is a useful package!')
    
from IPython.display import display, Latex, clear_output

In [2]:
# pyNRC and associates
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import (webbpsf, poppy, pix_noise, S)
from astropy.table import Table

pynrc.setup_logging('WARNING', verbose=False)

In [3]:
# Filters
# filt_wide_sw = ['F070W', 'F090W', 'F115W', 'F150W', 'F150W2', 'F200W']
# filt_wide_lw = ['F277W', 'F322W2', 'F356W', 'F444W']

# filt_med_sw = ['F140M', 'F162M', 'F182M', 'F210M']
filt_med_lw = ['F335M']

# filt_narr_sw = ['F164N', 'F187N', 'F212N']
# filt_narr_lw = ['F323N', 'F405N', 'F466N', 'F470N']

# filt_all_sw = filt_wide_sw + filt_med_sw + filt_narr_sw
# filt_all_lw = filt_wide_lw + filt_med_lw + filt_narr_lw
filt_all = filt_med_lw#filt_all_sw + filt_all_lw

In [4]:
read_modes_all = ['RAPID', 'BRIGHT1', 'BRIGHT2', 'SHALLOW2', 'SHALLOW4', 'MEDIUM2', 'MEDIUM8', 'DEEP2', 'DEEP8']
read_names_all = ['RAPID   ', 'BRIGHT1 ', 'BRIGHT2 ', 'SHALLOW2', 'SHALLOW4', 'MEDIUM2 ', 'MEDIUM8 ', 'DEEP2   ', 'DEEP8   ']

In [5]:
# NIRCam readout modes
# full   = {'wind_mode':'FULL',   'xpix':2048, 'ypix':2048}
# Window subarrays
sub32  = {'wind_mode':'WINDOW', 'xpix':32,   'ypix':32}

# sub160 = {'wind_mode':'WINDOW', 'xpix':160,  'ypix':160}
# sub320 = {'wind_mode':'WINDOW', 'xpix':320,  'ypix':320}
# sub400 = {'wind_mode':'WINDOW', 'xpix':400,  'ypix':400}
# sub640 = {'wind_mode':'WINDOW', 'xpix':640,  'ypix':640}

# Stripe subarrays
# stp64  = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':64}
# stp128 = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':128}
# stp256 = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':256}

subarray_settings = [sub32]

# Header names for each readout mode
head = []
head2 = []
for d in subarray_settings:
    if 'FULL' in d['wind_mode']:
        name = 'FULL'
        name2 = name
    elif 'WINDOW' in d['wind_mode']:
        name = 'SUB' + str(d['ypix'])
        name2 = name
    elif 'STRIPE' in d['wind_mode']:
        name = 'STRIPE' + str(d['ypix'])
        name2 = 'STRP' + str(d['ypix'])
    head.append(name)
    head2.append(name2)

In [16]:
nfilt = len(filt_all)
nsub  = len(subarray_settings)
sat_lims = np.zeros([nfilt,nsub])
sens_lims= np.zeros([nfilt,nsub])
# Module A
module = 'A'
nsig   = 30

ngroup = 3
units  = 'vegamag'
# We want the K-Band limiting magnitude for a G2V star
sp = nrc_utils.stellar_spectrum('G2V')
bp_lim = S.ObsBandpass('johnson,k')
bp_lim.name = 'K-Band'

filt = 'F335M'
# Loop through each subrray mode for a given filter and print the CDS saturation limit
modules = ['A','B']
tblsSat_dict = {}
tblsSens_dict = {}
for mod in modules:
    print("\nModule " + mod + ' SUB32')
    print('Read Mode\tSaturation\tSensitivity')
    for i, (read_mode, rm_name) in enumerate(zip(read_modes_all, read_names_all)):
        nrc = pynrc.NIRCam(filt, read_mode=read_mode, ngroup=ngroup, module=mod, **sub32)
        sat_lim = nrc.sat_limits(sp=sp, bp_lim=bp_lim, units=units)
        # sat_lims  = sat_lims['satmag']
        sens_lim = nrc.sensitivity(nsig=nsig, sp=sp, bp_lim=bp_lim, units=units)[0]
        # sens_lims = sens_lim['sensitivity']
        print(rm_name \
                                                     + '\t' + str(np.round(sat_lim['satmag'],2)) \
                                                     + '\t\t' + str(np.round(sens_lim['sensitivity'],2)))
        print()


Module A SUB32
Read Mode	Saturation	Sensitivity
RAPID   	7.23		11.85

BRIGHT1 	7.79		12.6

BRIGHT2 	7.98		12.84

SHALLOW2	8.74		13.84

SHALLOW4	8.9		14.03

MEDIUM2 	9.39		14.6

MEDIUM8 	9.66		14.93

DEEP2   	10.1		15.35

DEEP8   	10.24		15.68


Module B SUB32
Read Mode	Saturation	Sensitivity
RAPID   	7.12		11.74

BRIGHT1 	7.68		12.5

BRIGHT2 	7.87		12.74

SHALLOW2	8.63		13.74

SHALLOW4	8.79		13.93

MEDIUM2 	9.28		14.49

MEDIUM8 	9.55		14.82

DEEP2   	9.99		15.24

DEEP8   	10.13		15.58



In [24]:
nfilt = len(filt_all)
nsub  = len(subarray_settings)
sat_lims = np.zeros([nfilt,nsub])
sens_lims= np.zeros([nfilt,nsub])
# Module A
module = 'A'
nsig   = 10

ngroup = 3
units  = 'nJy'
# We want the K-Band limiting magnitude for a G2V star
sp = nrc_utils.stellar_spectrum('G2V')
bp_lim = S.ObsBandpass('johnson,k')
bp_lim.name = 'K-Band'

filt = 'F335M'
# Loop through each subrray mode for a given filter and print the CDS saturation limit
modules = ['A','B']
tblsSat_dict = {}
tblsSens_dict = {}
for mod in modules:
    print("\nModule " + mod + ' SUB32')
    print('Read Mode\tSaturation\tSensitivity')
    for i, (read_mode, rm_name) in enumerate(zip(read_modes_all, read_names_all)):
        nrc = pynrc.NIRCam(filt, read_mode=read_mode, ngroup=ngroup, module=mod, **sub32)
        sat_lim = nrc.sat_limits(sp=sp, bp_lim=bp_lim, units=units)
        # sat_lims  = sat_lims['satmag']
        sens_lim = nrc.sensitivity(nsig=nsig, sp=sp, bp_lim=bp_lim, units=units)[0]
        # sens_lims = sens_lim['sensitivity']
        print(rm_name, sens_lim)
        print()


Module A SUB32
Read Mode	Saturation	Sensitivity
RAPID    {'sensitivity': 1523061.1318796296, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

BRIGHT1  {'sensitivity': 760733.99471682974, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

BRIGHT2  {'sensitivity': 571564.14144725387, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

SHALLOW2 {'sensitivity': 228431.01609461516, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

SHALLOW4 {'sensitivity': 177605.55207438895, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

MEDIUM2  {'sensitivity': 114181.54192902167, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

MEDIUM8  {'sensitivity': 72378.932711765388, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

DEEP2    {'sensitivity': 57102.777304865311, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}

DEEP8    {'sensitivity': 36217.423061947149, 'units': 'nJy', 'nsig': 10, 'Spectrum': 'G2V'}


Module B SUB32
Read Mode	Saturation	Sensitivity
RAPID    {'sensitivity': 1668640.159205049, 'units': 'nJy', 'nsig